In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# show all columns in the df when printing to screen
pd.options.display.max_columns = None

In [2]:
# load csv file (200k results)
df = pd.read_csv("../petfinder_data/favorites_sample_small.csv", sep=";")

# remove all duplicate versions of a pet
df.drop_duplicates(subset=['id'], keep="first", inplace=True)
df

,user_id,favorited_at,id,organization_id,organization_active,organization_enabled,organization_display_id,import_on,import_software,organization_type,location_type,is_public_location,state_code,country_code,adoption_status,created_at,updated_at,name,organization_animal_identifier,tags,adoption_fee,adoption_fee_waived,display_adoption_fee,published_date,mixed_breed,unknown_breed,pet_description,pet_description_length,special_needs_notes,age,animal_type,primary_breed,secondary_breed,primary_color,secondary_color,tertiary_color,gender,size,species,coat_length,latitude,longitude,postal_code,good_with_children,good_with_dogs,good_with_cats,good_with_other_animals,good_with_others_notes,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_aifs,num_videos
0,4369629,2020-01-01 00:00:04,46861637,44719,1,1,OH1073,0,NaN,Rescue Group / Foster-Based,Default Location,0,OH,US,Adoptable,2019-12-16 15:06:18,2020-01-24 13:12:54,Elsa,NaN,NaN,NaN,0,0,2019-12-16 15:08:08,1,0,Meet Elsa!Elsa came to us as a stray. She had ...,639.0,DEAF,Baby,Dog,Pit Bull Terrier,NaN,NaN,NaN,NaN,Female,Medium,Dog,Short,41.457000,-81.787500,44111,unknown,yes,unknown,unknown,NaN,0,1,0,1,1,5,11,0
1,4369629,2020-01-01 00:00:07,46898514,35651,1,1,OH745,0,NaN,Rescue Group / Foster-Based,Default Location,1,OH,US,Adoptable,2019-12-20 02:58:49,2020-01-30 03:11:07,Coco,NaN,NaN,NaN,0,0,2019-12-20 03:02:07,1,0,Coco came into the rescue a couple years ago w...,945.0,NaN,Young,Dog,Pit Bull Terrier,NaN,NaN,NaN,NaN,Female,Medium,Dog,NaN,41.291000,-82.050900,44044,yes,yes,no,unknown,NaN,0,1,0,0,1,2,0,0
2,4112169,2020-01-01 00:00:08,46980809,40518,1,1,NY1044,0,NaN,Rescue Group / Foster-Based,Default Location,0,NY,US,Adopted,2019-12-31 18:24:35,2020-01-12 16:28:59,Cooper,NaN,NaN,NaN,0,0,2019-12-31 18:26:55,0,0,AGE: 1 YearSEX: Neutered MaleWEIGHT: 50 Pounds...,1005.0,NaN,Young,Dog,Labrador Retriever,NaN,NaN,NaN,NaN,Male,Medium,Dog,Medium,42.901100,-76.793400,13148,yes,yes,yes,unknown,NaN,1,1,0,0,1,6,0,0
3,4369680,2020-01-01 00:00:12,46956517,29207,1,1,FL806,0,NaN,NaN,Default Location,1,FL,US,Adoptable,2019-12-28 07:50:35,2020-01-01 20:18:10,Frankie,436,NaN,NaN,0,0,2019-12-28 07:53:26,1,0,Very sweet puppy will neuter micrchip and give...,151.0,NaN,Baby,Dog,German Shepherd Dog,NaN,NaN,NaN,NaN,Male,Large,Dog,Medium,25.773900,-80.193900,33153,yes,yes,yes,unknown,NaN,0,0,0,0,1,1,1,0
4,4369681,2020-01-01 00:00:13,46860620,48762,1,1,TN914,1,Petfinder Import Standard Template,Rescue Group / Foster-Based,Foster Home,0,CT,US,Adoptable,2019-12-16 07:30:36,2020-01-03 07:31:08,NaN,FFRR-A-36,NaN,NaN,0,0,2019-12-16 07:30:36,0,0,NaN,NaN,NaN,Baby,Dog,American Staffordshire Terrier,NaN,NaN,NaN,NaN,Female,Small,Dog,NaN,41.660100,-72.661400,06067,unknown,unknown,unknown,unknown,NaN,0,0,0,0,0,3,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140953,4260498,2020-01-05 02:43:16,47025142,26948,1,1,CA1110,1,Chameleon,Private Animal Shelter,Default Location,1,CA,US,Adoptable,2020-01-04 15:57:01,2020-01-08 23:09:03,NaN,A535103,NaN,NaN,0,0,2020-01-04 15:57:01,0,0,NaN,NaN,NaN,Baby,Dog,Plott Hound,NaN,NaN,NaN,NaN,Female,Small,Dog,NaN,37.644900,-120.969700,95354,unknown,unknown,unknown,unknown,NaN,0,0,0,0,0,1,1,0
140956,4397520,2020-01-05 02:43:17,46710925,43609,1,1,OH1043,0,NaN,Municipal Animal Shelter,Default Location,1,OH,US,Adopted,2019-11-29 05:04:03,2020-01-26 15:29:30,Walter,104,NaN,NaN,0,0,2019-11-29 05:05:40,1,0,Walter is a feisty young guy! If you're lookin...,2899.0,NaN,Young,Dog,German Shepherd Dog,Mixed Breed,Brown / Chocolate,White / Cream,NaN,Male,Medium,Dog,Short,41.473900,-81.737300,44102,unknown,unknown,unknown,unknown,NaN,1,1,0,0,1,5,3,0
140959,3980474,2020-01-05 02:43:19,47030490,27014,1,1,CA12,0,NaN,Rescue Group / Foster-Based,Default Location,0,CA,US,Adopted,2020-01-05 02:39:28,2020-01-10 03:1

In [3]:
# replace both 'pet_description' and 'pet_description_length' columns with a new column 'has_description'
def create_has_description(row):
    # compare on 'pet_description' field
    # the 'pet_description_length' field uses both 0.0 and nan for a pet with no description
    if pd.isnull(row['pet_description']):
        return 0
    else:
        return 1
df['has_description'] = df.apply(lambda row: create_has_description(row), axis=1)
# verify flag is working properly
df[['has_description', 'pet_description']]

,has_description,pet_description
0,1,Meet Elsa!Elsa came to us as a stray. She had ...
1,1,Coco came into the rescue a couple years ago w...
2,1,AGE: 1 YearSEX: Neutered MaleWEIGHT: 50 Pounds...
3,1,Very sweet puppy will neuter micrchip and give...
4,0,NaN
...,...,...
140953,0,NaN
140956,1,Walter is a feisty young guy! If you're lookin...
140959,1,NOTE: This is a courtesy posting for another r...
140961,1,"Please note Margo is located in Denver, Colora..."


In [4]:
# drop columns that the model will not train on
to_drop = [
    'adoption_fee',
    'adoption_fee_waived',
    'adoption_status',
    'country_code',
    'created_at',
    'display_adoption_fee',
    'favorited_at',
    'good_with_others_notes',
    'id',
    'import_on',
    'import_software',
    'is_public_location',
    'latitude',
    'location_type',
    'longitude',
    'name',
    'num_aifs',
    'organization_active',
    'organization_animal_identifier',
    'organization_display_id',
    'organization_enabled',
    'organization_id',
    'organization_type',
    'pet_description',
    'pet_description_length',
    'postal_code',
    'published_date',
    'secondary_breed',
    'secondary_color',
    'special_needs_notes',
    'species',
    'state_code',
    'tags',
    'tertiary_color',
    'unknown_breed',
    'updated_at',
    'user_id'
]
df.drop(to_drop, axis=1, inplace=True)

In [5]:
# fill nan values in all color fields
df['primary_color'].fillna(value='NO_PRIMARY_COLOR', inplace=True)
# fill nan values in for coat_length
df['coat_length'].fillna(value='NO_COAT_LENGTH', inplace=True)
df.head()

,mixed_breed,age,animal_type,primary_breed,primary_color,gender,size,coat_length,good_with_children,good_with_dogs,good_with_cats,good_with_other_animals,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_videos,has_description
0,1,Baby,Dog,Pit Bull Terrier,NO_PRIMARY_COLOR,Female,Medium,Short,unknown,yes,unknown,unknown,0,1,0,1,1,5,0,1
1,1,Young,Dog,Pit Bull Terrier,NO_PRIMARY_COLOR,Female,Medium,NO_COAT_LENGTH,yes,yes,no,unknown,0,1,0,0,1,2,0,1
2,0,Young,Dog,Labrador Retriever,NO_PRIMARY_COLOR,Male,Medium,Medium,yes,yes,yes,unknown,1,1,0,0,1,6,0,1
3,1,Baby,Dog,German Shepherd Dog,NO_PRIMARY_COLOR,Male,Large,Medium,yes,yes,yes,unknown,0,0,0,0,1,1,0,1
4,0,Baby,Dog,American Staffordshire Terrier,NO_PRIMARY_COLOR,Female,Small,NO_COAT_LENGTH,unknown,unknown,unknown,unknown,0,0,0,0,0,3,0,0


In [6]:
# validate all columns have 63,307 non-null values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63307 entries, 0 to 140963
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   mixed_breed              63307 non-null  int64 
 1   age                      63307 non-null  object
 2   animal_type              63307 non-null  object
 3   primary_breed            63307 non-null  object
 4   primary_color            63307 non-null  object
 5   gender                   63307 non-null  object
 6   size                     63307 non-null  object
 7   coat_length              63307 non-null  object
 8   good_with_children       63307 non-null  object
 9   good_with_dogs           63307 non-null  object
 10  good_with_cats           63307 non-null  object
 11  good_with_other_animals  63307 non-null  object
 12  house_trained            63307 non-null  int64 
 13  altered                  63307 non-null  int64 
 14  declawed                 63307 non-nu

In [7]:
# convert all 'good_with_xxx columns' to 0 or 1 (include unknown with 0)
# replace both 'pet_description' and 'pet_description_length' columns with a new column 'has_description'
def convert_to_bin(row, col):
    # compare on 'pet_description' field
    # the 'pet_description_length' field uses both 0.0 and nan for a pet with no description
    if row[col] in ["no", "unknown"]:
        return 0
    else:
        return 1
df['good_with_children'] = df.apply(lambda row: convert_to_bin(row, "good_with_children"), axis=1)
df['good_with_dogs'] = df.apply(lambda row: convert_to_bin(row, "good_with_dogs"), axis=1)
df['good_with_cats'] = df.apply(lambda row: convert_to_bin(row, "good_with_cats"), axis=1)
df['good_with_other_animals'] = df.apply(lambda row: convert_to_bin(row, "good_with_other_animals"), axis=1)
df.head()

,mixed_breed,age,animal_type,primary_breed,primary_color,gender,size,coat_length,good_with_children,good_with_dogs,good_with_cats,good_with_other_animals,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_videos,has_description
0,1,Baby,Dog,Pit Bull Terrier,NO_PRIMARY_COLOR,Female,Medium,Short,0,1,0,0,0,1,0,1,1,5,0,1
1,1,Young,Dog,Pit Bull Terrier,NO_PRIMARY_COLOR,Female,Medium,NO_COAT_LENGTH,1,1,0,0,0,1,0,0,1,2,0,1
2,0,Young,Dog,Labrador Retriever,NO_PRIMARY_COLOR,Male,Medium,Medium,1,1,1,0,1,1,0,0,1,6,0,1
3,1,Baby,Dog,German Shepherd Dog,NO_PRIMARY_COLOR,Male,Large,Medium,1,1,1,0,0,0,0,0,1,1,0,1
4,0,Baby,Dog,American Staffordshire Terrier,NO_PRIMARY_COLOR,Female,Small,NO_COAT_LENGTH,0,0,0,0,0,0,0,0,0,3,0,0


In [8]:
# encode 'age' column manually since we want a speficic order
# order: ["Baby", "Young", "Adult", "Senior"]

def encode_age(row):
    if row["age"] == "Baby":
        return 0
    elif row["age"] == "Young":
        return 1
    elif row["age"] == "Adult":
        return 2
    else:
        # senior pets
        return 3

df['age'] = df.apply(lambda row: encode_age(row), axis=1)
df.head()

,mixed_breed,age,animal_type,primary_breed,primary_color,gender,size,coat_length,good_with_children,good_with_dogs,good_with_cats,good_with_other_animals,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_videos,has_description
0,1,0,Dog,Pit Bull Terrier,NO_PRIMARY_COLOR,Female,Medium,Short,0,1,0,0,0,1,0,1,1,5,0,1
1,1,1,Dog,Pit Bull Terrier,NO_PRIMARY_COLOR,Female,Medium,NO_COAT_LENGTH,1,1,0,0,0,1,0,0,1,2,0,1
2,0,1,Dog,Labrador Retriever,NO_PRIMARY_COLOR,Male,Medium,Medium,1,1,1,0,1,1,0,0,1,6,0,1
3,1,0,Dog,German Shepherd Dog,NO_PRIMARY_COLOR,Male,Large,Medium,1,1,1,0,0,0,0,0,1,1,0,1
4,0,0,Dog,American Staffordshire Terrier,NO_PRIMARY_COLOR,Female,Small,NO_COAT_LENGTH,0,0,0,0,0,0,0,0,0,3,0,0


In [9]:
# encode 'animal_type' column
animal_type_enc = LabelEncoder()
# can later use animal_type_enc like so animal_type_enc.inverse_transform([2]) to get original type
df['animal_type'] = animal_type_enc.fit_transform(df['animal_type'].values)

# encode 'primary_breed' column
breed_enc = LabelEncoder()
df['primary_breed'] = breed_enc.fit_transform(df['primary_breed'].values)

# encode 'primary_color' column
color_enc = LabelEncoder()
df['primary_color'] = color_enc.fit_transform(df['primary_color'].values)

# encode 'gender' column
gender_enc = LabelEncoder()
df['gender'] = gender_enc.fit_transform(df['gender'].values)

# encode 'coat_length' column
coat_length_enc = LabelEncoder()
df['coat_length'] = coat_length_enc.fit_transform(df['coat_length'].values)

df.head()

,mixed_breed,age,animal_type,primary_breed,primary_color,gender,size,coat_length,good_with_children,good_with_dogs,good_with_cats,good_with_other_animals,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_videos,has_description
0,1,0,3,292,48,0,Medium,5,0,1,0,0,0,1,0,1,1,5,0,1
1,1,1,3,292,48,0,Medium,4,1,1,0,0,0,1,0,0,1,2,0,1
2,0,1,3,218,48,1,Medium,3,1,1,1,0,1,1,0,0,1,6,0,1
3,1,0,3,169,48,1,Large,3,1,1,1,0,0,0,0,0,1,1,0,1
4,0,0,3,20,48,0,Small,4,0,0,0,0,0,0,0,0,0,3,0,0


In [10]:
# encode 'size' column manually since we want a speficic order
# order: ["Baby", "Young", "Adult", "Senior"]

def encode_size(row):
    if row["size"] == "Small":
        return 0
    elif row["size"] == "Medium":
        return 1
    elif row["size"] == "Large":
        return 2
    else:
        # Extra Large Pets
        return 3

df['size'] = df.apply(lambda row: encode_size(row), axis=1)
df.head()

,mixed_breed,age,animal_type,primary_breed,primary_color,gender,size,coat_length,good_with_children,good_with_dogs,good_with_cats,good_with_other_animals,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_videos,has_description
0,1,0,3,292,48,0,1,5,0,1,0,0,0,1,0,1,1,5,0,1
1,1,1,3,292,48,0,1,4,1,1,0,0,0,1,0,0,1,2,0,1
2,0,1,3,218,48,1,1,3,1,1,1,0,1,1,0,0,1,6,0,1
3,1,0,3,169,48,1,2,3,1,1,1,0,0,0,0,0,1,1,0,1
4,0,0,3,20,48,0,0,4,0,0,0,0,0,0,0,0,0,3,0,0


In [11]:
print (len(df), "pets")
print (len(df.columns), "features")

63307 pets
20 features


In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

# convert each row of the dataframe into an array
pets = df.values

def input_fn():
  return tf.compat.v1.train.limit_epochs(
      tf.convert_to_tensor(pets, dtype=tf.float32), num_epochs=10)

num_clusters = 10
kmeans = tf.compat.v1.estimator.experimental.KMeans(
    num_clusters=num_clusters, use_mini_batch=False)

# train
num_iterations = 10
previous_centers = None
for _ in range(num_iterations):
    kmeans.train(input_fn)
    cluster_centers = kmeans.cluster_centers()
    if previous_centers is not None:
        print('DELTA:', cluster_centers - previous_centers)
    previous_centers = cluster_centers
    print('SCORE:', kmeans.score(input_fn))
    print("--------FINISHED 1 ITERATION--------")
print ('final cluster centers:', cluster_centers)

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_re

  -1.0974407e-03  2.2909641e-03  1.3997406e-04 -8.1908703e-04]]
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-10T13:41:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-21
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Inference Time : 0.21454s
INFO:tensorflow:Finished evaluation at 2020-07-10-13:41:41
INFO:tensorflow:Saving dict for global step 21: global_step = 21, loss = 33014864.0, score = 33014864.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21: /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-21
SCORE: 33014864.0
--------FINISHED 1 ITERATION--------
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorf

DELTA: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-10T13:41:46Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-54
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_i

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Inference Time : 0.20474s
INFO:tensorflow:Finished evaluation at 2020-07-10-13:41:52
INFO:tensorflow:Saving dict for global step 87: global_step = 87, loss = 33014864.0, score = 33014864.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 87: /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-87
SCORE: 33014864.0
--------FINISHED 1 ITERATION--------
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-87
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 87...
INFO:tensorflow:Saving checkpoints for 87 into /va

In [13]:
# map each pet to their clusters
cluster_indices = list(kmeans.predict_cluster_index(input_fn))
output = {
    "0": {"breeds": {}},
    "1": {"breeds": {}},
    "2": {"breeds": {}},
    "3": {"breeds": {}},
    "4": {"breeds": {}},
    "5": {"breeds": {}},
    "6": {"breeds": {}},
    "7": {"breeds": {}},
    "8": {"breeds": {}},
    "9": {"breeds": {}},
}
flag = True
for i, pet_data in enumerate(pets):
    cluster = str(cluster_indices[i])
    breed = breed_enc.inverse_transform([pet_data[3]])[0]
    # add to output dictionary
    if breed in output[cluster]["breeds"]: 
        output[cluster]["breeds"][breed] += 1
    else:
        output[cluster]["breeds"][breed] = 1
    if flag:
        # print out the first pet's prediction so it can be used to test the model in React
        print (pet_data)
        print (cluster)
        flag = False

import json
with open('clusters.json', 'w') as fp:
    json.dump(output, fp)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-109
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[  1   0   3 292  48   0   1   5   0   1   0   0   0   1   0   1   1   5
   0   1]
8


In [15]:
def serving_input_receiver_fn():
    feature_placeholders = {
        'mixed_breed' : tf.placeholder(tf.float32, [1]),
        'age' : tf.placeholder(tf.float32, [1]),
        'animal_type' : tf.placeholder(tf.float32, [1]),
        'primary_breed' : tf.placeholder(tf.float32, [1]),
        'primary_color' : tf.placeholder(tf.float32, [1]),
        'gender' : tf.placeholder(tf.float32, [1]),
        'size' : tf.placeholder(tf.float32, [1]),
        'coat_length' : tf.placeholder(tf.float32, [1]),
        'good_with_children' : tf.placeholder(tf.float32, [1]),
        'good_with_dogs' : tf.placeholder(tf.float32, [1]),
        'good_with_cats' : tf.placeholder(tf.float32, [1]),
        'good_with_other_animals' : tf.placeholder(tf.float32, [1]),
        'house_trained' : tf.placeholder(tf.float32, [1]),
        'altered' : tf.placeholder(tf.float32, [1]),
        'declawed' : tf.placeholder(tf.float32, [1]),
        'special_needs' : tf.placeholder(tf.float32, [1]),
        'shots_current' : tf.placeholder(tf.float32, [1]),
        'num_photos' : tf.placeholder(tf.float32, [1]),
        'num_videos' : tf.placeholder(tf.float32, [1]),
        'has_description' : tf.placeholder(tf.float32, [1]),
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, 
                                                    feature_placeholders)

kmeans.export_saved_model(export_dir_base="./saved_model", serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['all_distances', 'cluster_index', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /var/folders/2_/_8xm9cb17rx3dn7qvpx_9g6c0000gp/T/tmp8jto0qr7/model.ckpt-109
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/temp-1594408565/saved_model.pb


b'./saved_model/1594408565'